In [1]:
import numpy as np
import h5py
import tensorflow as tf
import pandas as pd

In [2]:
def split(array_2d,nb):
    array_2d = np.array(np.split(array_2d, nb, axis=1))
    return array_2d.reshape(-1, array_2d.shape[-1])

In [3]:
h5_datas = h5py.File("../ml_project_data/h5/X_train.h5/X_train.h5","r")
#h5_datas = h5py.File("F:\ML_Project\dreem-2-sleep-classification-challenge-2020\X_train.h5\X_train.h5","r")


eeg_1 = split(h5_datas["eeg_1"][...],2)
eeg_2 = split(h5_datas["eeg_2"][...],2)
eeg_3 = split(h5_datas["eeg_3"][...],2)
eeg_4 = split(h5_datas["eeg_4"][...],2)
eeg_5 = split(h5_datas["eeg_5"][...],2)
eeg_6 = split(h5_datas["eeg_6"][...],2)
eeg_7 = split(h5_datas["eeg_7"][...],2)
pulse = np.repeat(split(h5_datas["pulse"][...],2),5,axis=1)
x =  np.repeat(split(h5_datas["x"][...],2),5,axis=1)
y =  np.repeat(split(h5_datas["y"][...],2),5,axis=1)
z = np.repeat(split(h5_datas["z"][...],2),5,axis=1)
labels =  np.tile(pd.read_csv('../ml_project_data/h5/y_train.csv').values[:,1],1)
#labels =  np.tile(pd.read_csv('F:\ML_Project\dreem-2-sleep-classification-challenge-2020\y_train.csv').values[:,1],6)

h5_datas.close()

In [4]:
len(labels)/16

24688

In [5]:
def get_observation(observation_id=0):
    observation_data = {
            'size': eeg_1[observation_id].shape[0],
            'eeg_1': eeg_1[observation_id],
            'eeg_2': eeg_2[observation_id],
            'eeg_3': eeg_3[observation_id],
            'eeg_4': eeg_4[observation_id],
            'eeg_5': eeg_5[observation_id],
            'eeg_5': eeg_5[observation_id],
            'eeg_6': eeg_6[observation_id],
            'eeg_7': eeg_7[observation_id],
            'pulse': pulse[observation_id],
            'x': x[observation_id],
            'y': y[observation_id],
            'z': y[observation_id],
            'label': labels[observation_id]
            }
    return observation_data


In [6]:
def get_example_object(single_record):

    record = tf.train.Example(features=tf.train.Features(feature={
        'size': tf.train.Feature(
            int64_list=tf.train.Int64List(value=[single_record['size']])),
        'eeg_1': tf.train.Feature(
            float_list=tf.train.FloatList(value=single_record['eeg_1'])), 
            #bytes_list=tf.train.BytesList(value=[single_record['img_string'].tobytes(order='C')])),
        'eeg_2': tf.train.Feature(
            float_list=tf.train.FloatList(value=single_record['eeg_2'])), 
            #bytes_list=tf.train.BytesList(value=[single_record['img_string'].tobytes(order='C')])),
        'eeg_3': tf.train.Feature(
            float_list=tf.train.FloatList(value=single_record['eeg_3'])), 
            #bytes_list=tf.train.BytesList(value=[single_record['img_string'].tobytes(order='C')])),
        'eeg_4': tf.train.Feature(
            float_list=tf.train.FloatList(value=single_record['eeg_4'])), 
            #bytes_list=tf.train.BytesList(value=[single_record['img_string'].tobytes(order='C')])),
        'eeg_5': tf.train.Feature(
            float_list=tf.train.FloatList(value=single_record['eeg_5'])), 
            #bytes_list=tf.train.BytesList(value=[single_record['img_string'].tobytes(order='C')])),
        'eeg_6': tf.train.Feature(
            float_list=tf.train.FloatList(value=single_record['eeg_6'])), 
            #bytes_list=tf.train.BytesList(value=[single_record['img_string'].tobytes(order='C')])),
        'eeg_7': tf.train.Feature(
            float_list=tf.train.FloatList(value=single_record['eeg_7'])), 
            #bytes_list=tf.train.BytesList(value=[single_record['img_string'].tobytes(order='C')])),
        'pulse': tf.train.Feature(
            float_list=tf.train.FloatList(value=single_record['pulse'])), 
            #bytes_list=tf.train.BytesList(value=[single_record['img_string'].tobytes(order='C')])),
        'x': tf.train.Feature(
            float_list=tf.train.FloatList(value=single_record['x'])), 
            #bytes_list=tf.train.BytesList(value=[single_record['img_string'].tobytes(order='C')])),
        'y': tf.train.Feature(
            float_list=tf.train.FloatList(value=single_record['y'])), 
            #bytes_list=tf.train.BytesList(value=[single_record['img_string'].tobytes(order='C')])),
        'z': tf.train.Feature(
            float_list=tf.train.FloatList(value=single_record['z'])), 
            #bytes_list=tf.train.BytesList(value=[single_record['img_string'].tobytes(order='C')])),
        'label': tf.train.Feature(
            int64_list=tf.train.Int64List(value=[single_record['label']]))
    }))
    return record


In [ ]:
#Number of TFR files to save the data into
numFiles = 16
records_per_file = int(len(labels)/numFiles)
num=0
for file_id in range(numFiles):
    num+=1
    #with tf.io.TFRecordWriter('F:\ML_Project\Records\SIGNALS_train_datastrings' + str(file_id) + '.tfrecord') as tfwriter:
    with tf.io.TFRecordWriter('../ml_project_data/records_tf/SIGNALS_train_data_strings_' + str(file_id) + '.tfrecord') as tfwriter:
        # Iterate through all records
        for observation_id in range(records_per_file):
            observation_data = get_observation(observation_id + file_id * records_per_file)
            example = get_example_object(observation_data)
    
            # Append each record into TFRecord
            tfwriter.write(example.SerializeToString())
        print("fichier n°",num,"écrit")
            
